In [ ]:
!pip install evaluate

In [ ]:
# Environment and configuration
import os
import time
import random
import math

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from transformers import (
    T5TokenizerFast,
    TFT5ForConditionalGeneration,
    create_optimizer,
)

import datasets as hf_datasets
import evaluate as hf_evaluate

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Paths
DATA_PATH = os.path.join(os.getcwd(), 'bitext-mortgage-loans-llm-chatbot-training-dataset.csv')
SAVE_ROOT = os.path.join(os.getcwd(), 'saved_models')
os.makedirs(SAVE_ROOT, exist_ok=True)

# Model and training hyperparameters
MODEL_NAME = 'google/flan-t5-small'
MAX_SOURCE_LENGTH = 256
MAX_TARGET_LENGTH = 128
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 3e-4
WARMUP_RATIO = 0.06
WEIGHT_DECAY = 0.01

RUN_ID = time.strftime('HUFI_V1_FLAN_T5_%Y%m%d_%H%M%S')
OUTPUT_DIR = os.path.join(SAVE_ROOT, RUN_ID)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'Run ID: {RUN_ID}\nSaving to: {OUTPUT_DIR}')


Run ID: HUFI_V1_FLAN_T5_20251013_141421
Saving to: /content/saved_models/HUFI_V1_FLAN_T5_20251013_141421


In [ ]:
# Load and inspect dataset
assert os.path.exists(DATA_PATH), f"Dataset not found at {DATA_PATH}"

try:
    df = pd.read_csv(DATA_PATH, engine='python', on_bad_lines='skip')
except Exception as e:
    print(f"Error reading CSV: {e}")
    # As a fallback, try reading with a different delimiter if applicable
    # For example, if it might be tab-separated:
    # df = pd.read_csv(DATA_PATH, sep='\t', engine='python', on_bad_lines='skip')
    # Or if it's a different common delimiter
    # df = pd.read_csv(DATA_PATH, sep=';', engine='python', on_bad_lines='skip')
    raise  # Re-raise the exception if reading still fails

print(df.head(2))
print('Columns:', df.columns.tolist())
print('Shape:', df.shape)

# Identify columns
QUESTION_COL = 'instruction' if 'instruction' in df.columns else df.columns[1]
ANSWER_COL = 'response' if 'response' in df.columns else df.columns[-1]

# Clean basic
for col in [QUESTION_COL, ANSWER_COL]:
    df[col] = df[col].astype(str).str.strip()

df = df.dropna(subset=[QUESTION_COL, ANSWER_COL])
df = df.drop_duplicates(subset=[QUESTION_COL, ANSWER_COL])
print('After cleaning:', df.shape)

                                       system_prompt  \
0  You are an expert in customer support for mort...   
1  You are an expert in customer support for mort...   

                                         instruction          intent  \
0  could you help me to add a co-borrower to my m...  add_coborrower   
1  I would like to add a co-borrower ot my auto l...  add_coborrower   

             category    tags  \
0  LOAN_MODIFICATIONS    BILP   
1  LOAN_MODIFICATIONS  BCILPZ   

                                            response  
0  I'm on it! I'm here to assist you with adding ...  
1  Absolutely! I'm here to assist you in adding a...  
Columns: ['system_prompt', 'instruction', 'intent', 'category', 'tags', 'response']
Shape: (31038, 6)
After cleaning: (31038, 6)


In [ ]:
# Stratified train/val/test split by intent if available
from sklearn.model_selection import train_test_split

stratify_col = df['intent'] if 'intent' in df.columns else None
train_df, test_df = train_test_split(df, test_size=0.1, random_state=SEED, shuffle=True, stratify=stratify_col)
stratify_col_tv = train_df['intent'] if 'intent' in train_df.columns else None
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=SEED, shuffle=True, stratify=stratify_col_tv)

print('Split sizes -> train:', len(train_df), 'val:', len(val_df), 'test:', len(test_df))


Split sizes -> train: 25140 val: 2794 test: 3104


In [ ]:
# Tokenizer and formatting

tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
PREFIX = 'answer the question: '

def format_example(question: str, answer: str):
    return PREFIX + question, answer

for i in range(2):
    s, t = format_example(train_df.iloc[i][QUESTION_COL], train_df.iloc[i][ANSWER_COL])
    print('SRC:', s[:100])
    print('TGT:', t[:100])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

SRC: answer the question: help me checking the borrowing limit
TGT: Assuredly! I'd be more than happy to assist you with checking the borrowing limit. The borrowing lim
SRC: answer the question: I'd like to speak with an agent where could i do it
TGT: I regret that any inconvenience you may have experienced. I understand that sometimes it's necessary


In [ ]:
# Build Hugging Face datasets

def to_hf_dataset(frame: pd.DataFrame) -> hf_datasets.Dataset:
    sources, targets = [], []
    for _, row in frame.iterrows():
        s, t = format_example(row[QUESTION_COL], row[ANSWER_COL])
        sources.append(s)
        targets.append(t)
    return hf_datasets.Dataset.from_dict({'source': sources, 'target': targets})

raw_train = to_hf_dataset(train_df)
raw_val = to_hf_dataset(val_df)
raw_test = to_hf_dataset(test_df)

print(raw_train[0])


{'source': 'answer the question: help me checking the borrowing limit', 'target': 'Assuredly! I\'d be more than happy to assist you with checking the borrowing limit. The borrowing limit is the maximum amount that you can borrow from us. It is based on various factors such as your credit history, income, and existing debts. To check your borrowing limit, you can:\n\n1. Log in to your account on our website/app.\n2. Navigate to the "Account" or "Profile" section.\n3. Look for the "Borrowing Limit" or "Credit Limit" option.\n4. Click on it to view your current borrowing limit. \n\nIf you\'re unable to find the option or have any specific questions regarding your borrowing limit, feel free to reach out to our customer support team. They can provide personalized assistance and guide you through the process.'}


In [ ]:
# Tokenize datasets

def tokenize_function(batch):
    model_inputs = tokenizer(
        batch['source'],
        max_length=MAX_SOURCE_LENGTH,
        truncation=True,
        padding='max_length',
        return_tensors='np',
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['target'],
            max_length=MAX_TARGET_LENGTH,
            truncation=True,
            padding='max_length',
            return_tensors='np',
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_tokenized = raw_train.map(tokenize_function, batched=True, remove_columns=['source','target'])
val_tokenized = raw_val.map(tokenize_function, batched=True, remove_columns=['source','target'])
test_tokenized = raw_test.map(tokenize_function, batched=True, remove_columns=['source','target'])

for ds in [train_tokenized, val_tokenized, test_tokenized]:
    ds.set_format(type='numpy')


Map:   0%|          | 0/25140 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2794 [00:00<?, ? examples/s]

Map:   0%|          | 0/3104 [00:00<?, ? examples/s]

In [ ]:
# tf.data input pipelines

def to_tf_dataset(tokenized: hf_datasets.Dataset, batch_size: int) -> tf.data.Dataset:
    feats = {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],
        'labels': tokenized['labels'],
    }
    def gen():
        for i in range(len(tokenized)):
            yield {k: feats[k][i] for k in feats}
    sig = {
        'input_ids': tf.TensorSpec(shape=(None,), dtype=tf.int32),
        'attention_mask': tf.TensorSpec(shape=(None,), dtype=tf.int32),
        'labels': tf.TensorSpec(shape=(None,), dtype=tf.int32),
    }
    return tf.data.Dataset.from_generator(gen, output_signature=sig).shuffle(1024, seed=SEED).batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_ds = to_tf_dataset(train_tokenized, BATCH_SIZE)
val_ds = to_tf_dataset(val_tokenized, BATCH_SIZE)


In [ ]:
# Model and training loop

tf.keras.backend.clear_session()
model = TFT5ForConditionalGeneration.from_pretrained(MODEL_NAME, from_pt=True)

num_train_steps = math.ceil(len(train_tokenized) / BATCH_SIZE) * EPOCHS
num_warmup_steps = int(num_train_steps * WARMUP_RATIO)

optimizer, lr_schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_warmup_steps=num_warmup_steps,
    num_train_steps=num_train_steps,
    weight_decay_rate=WEIGHT_DECAY,
)

model.compile(optimizer=optimizer)

# Change OUTPUT_DIR to a path in Google Drive
OUTPUT_DIR = os.path.join(SAVE_ROOT, RUN_ID)
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'Run ID: {RUN_ID}\nSaving to: {OUTPUT_DIR}')

ckpt_cb = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(OUTPUT_DIR, 'ckpt'),
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
)

es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[ckpt_cb, es_cb],
)

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print('Saved to', OUTPUT_DIR)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
/usr/local/lib/python3.12/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected

Run ID: HUFI_V1_FLAN_T5_20251013_141421
Saving to: /content/saved_models/HUFI_V1_FLAN_T5_20251013_141421
Epoch 1/3
3143/3143 [==============================] - 980s 302ms/step - loss: 1.2528 - val_loss: 0.7844
Epoch 2/3
3143/3143 [==============================] - 943s 299ms/step - loss: 0.8518 - val_loss: 0.7089
Epoch 3/3
3143/3143 [==============================] - 946s 301ms/step - loss: 0.7819 - val_loss: 0.6828
Saved to /content/saved_models/HUFI_V1_FLAN_T5_20251013_141421


In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9d396c59a444e2d2a7b4cf95410fdea8b5e813f17195b08d5269cebcbbdef27b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
# Evaluation: BLEU, ROUGE-L, perplexity

bleu = hf_evaluate.load('sacrebleu')
rouge = hf_evaluate.load('rouge')

def generate_answers(questions, max_new_tokens=64):
    inputs = tokenizer(['answer the question: ' + q for q in questions], return_tensors='tf', padding=True, truncation=True, max_length=MAX_SOURCE_LENGTH)
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

sample_size = min(256, len(val_df))
sample_questions = val_df[QUESTION_COL].tolist()[:sample_size]
sample_refs = [[a] for a in val_df[ANSWER_COL].tolist()[:sample_size]]

preds = generate_answers(sample_questions)

bleu_res = bleu.compute(predictions=preds, references=sample_refs)
rouge_res = rouge.compute(predictions=preds, references=[r[0] for r in sample_refs])

val_loss = model.evaluate(val_ds, return_dict=True)['loss']
perplexity = math.exp(val_loss) if val_loss < 20 else float('inf')

print('BLEU:', bleu_res)
print('ROUGE-L:', rouge_res.get('rougeL'))
print('Val loss:', val_loss, 'Perplexity:', perplexity)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


350/350 [==============================] - 39s 107ms/step - loss: 0.6820
BLEU: {'score': 2.8819816181639233, 'counts': [10713, 6943, 5027, 3722], 'totals': [14150, 13894, 13638, 13382], 'precisions': [75.71024734982332, 49.97121059450122, 36.86024343745417, 27.81348079509789], 'bp': 0.06494103216020367, 'sys_len': 14150, 'ref_len': 52840}
ROUGE-L: 0.28295411427809747
Val loss: 0.6819899082183838 Perplexity: 1.977809478361478


In [ ]:
# Qualitative test: predictions vs references on a small sample

NUM_EXAMPLES = 10
MAX_NEW_TOKENS = 64

sample_questions = val_df[QUESTION_COL].tolist()[:NUM_EXAMPLES]
sample_refs = val_df[ANSWER_COL].tolist()[:NUM_EXAMPLES]

def generate_answers_list(questions, max_new_tokens=64):
    inputs = tokenizer(
        ['answer the question: ' + q for q in questions],
        return_tensors='tf',
        padding=True,
        truncation=True,
        max_length=MAX_SOURCE_LENGTH,
    )
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

preds = generate_answers_list(sample_questions, max_new_tokens=MAX_NEW_TOKENS)

for i, (q, ref, pred) in enumerate(zip(sample_questions, sample_refs, preds), 1):
    print(f'--- Example {i} ---')
    print('Q:', q)
    print('REF:', ref[:400])
    print('PRED:', pred[:400])
    print()

print(f'Shown {NUM_EXAMPLES} qualitative examples.')


--- Example 1 ---
Q: I got to modify my preferred account number, where can I do it?
REF: Certainly, I'm here to help you with modifying your preferred account number. Here's what you need to do:

1. Log in to your account on our website or mobile app.
2. Navigate to the "{{Account Settings}}" or "{{Profile}}" section.
3. Look for the option to update your account information.
4. You should find a field specifically for your preferred account number.
5. Enter the new account number you
PRED: Certainly, I can assist you with modifying your preferred account number. Here's what you need to do: 1. Log in to your account on our website or mobile app. 2. Navigate to the "Account Settings" or "Profile" section. 3. Look for the option to manage your preferred account number

--- Example 2 ---
Q: can ya help me to see my fucking home loan details
REF: We're accountable for, and we apologize for any frustration caused by the language used in your previous message. I'd be happy to assist you wit